<a href="https://colab.research.google.com/github/baker371/Drug-Discovery/blob/main/OpenSEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing Libraries

import re
import pandas as pd
import requests
import warnings
import json
import urllib.request

warnings.filterwarnings('ignore')

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

In [2]:
pro = pd.read_csv (r'rarity_tools_collections.csv')
pro.head(5)

,Unnamed: 0,Name,Collection Name,Supply,Contract,Unnamed: 5
0,0,Meebits,meebits,19999,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,f
1,1,CryptoPunks,cryptopunks,9999,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,f
2,2,Cupcats Official,cupcatsofficial,5007,0x8cd8155e1af6ad31dd9eec2ced37e04145acfcb3,f
3,3,ApeOnly Island,apeonly-island,2914,0x260428e36989ee6c6829f8a6e361cba99c7a8447,f
4,4,Capsule House,capsulehouse,10000,0xfcb1315c4273954f74cb16d5b663dbf479eec62e,f


In [8]:
pro = pd.read_csv (r'rarity_tools_collections.csv')
olist= pro[['Contract','Supply']].values.tolist()
nlist = olist[19:20]
nlist

[['0x375ea781c49eafedde07afe6196f885761f166ae', 23497]]

In [2]:
#@title

def flatten_json(nested_json: dict, exclude: list=['']) -> dict:
    """
    Flatten a list of nested dicts.
    """
    out = dict()
    def flatten(x: (list, dict, str), name: str='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude:
                    flatten(x[a], f'{name}{a}_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, f'{name}{i}_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [9]:
nlist = [['0x42f1654b8eeb80c96471451b1106b63d0b1a9fe1',4], ['0x990ee1df4c3047a4c94aac91a86be3c6bf40097d',2], ['0xcf86a1122cb1540f3183ad5b1e8a488a4eae7f3b',5]]

In [11]:
import numpy as np


for i in range(len(nlist)):
  mylist = [nlist[i]]
  items = []
  for address, count in mylist:
    num = 0
    while num <= count:
      num += 1
      url  = f'https://api.opensea.io/api/v1/assets?asset_contract_address={address}&token_ids={num}'
      items.append(url)
  df = []
  for link in items:
    req          = requests.get(link)
    jsondata     = json.loads(req.text)['assets'][0]
    id           = jsondata['token_id']
    slug         = jsondata['collection']['slug']
    traits       = jsondata['traits']
    dfo          = pd.json_normalize(traits)
    dfo1         = dfo[['value','trait_count']]
    dfo2         = dfo1.T
    dfo2         = dfo2.reset_index(drop=True)
    dfo2.columns = dfo2.iloc[0]
    dfo2['id']    = id
    dfo2['slug']    = slug
    dfn          = dfo2.drop(dfo2.index[0])
    df.append(dfn)
  dfo             = pd.concat(df) 
  dfo             = dfo[dfo['id'].notna()]
  count           = len(dfo)
  wanted          = dfo.drop(['slug', 'id'], axis=1)
  traits          = wanted.fillna(count - wanted.mean())
  trait_freq      = (traits/count).round(2)
  trait_rarity    = (1/trait_freq).round(2)
  trait_rarity    = trait_rarity.replace([np.inf, -np.inf], np.nan)
  cols_to_sum     = list(trait_rarity)
  traits['rarity_score'] = trait_rarity[cols_to_sum].sum(axis = 1).round(2)
  dfo['link'] =  "https://rarity.tools/"+ dfo.slug + "/view/" + dfo.id
  dfo['method'] = "apparent.tools v2"
  dfo['rarity_score'] =  traits['rarity_score']
  final = dfo[['slug','id','link','rarity_score','method']]
  final   = final.sort_values(['rarity_score'], ascending=[False])
  final.to_csv(slug +".csv", sep=',', index = True, header=True)
    

In [ ]:
headers = {"X-API-KEY": "a5a1917916b14c32a9804befa8e4388a"}
dfn = []
for item in col_list:
  url = f'https://api.opensea.io/api/v1/collection/{item}'
  response = requests.request("GET", url, headers=headers)
  data = response.json() 
  dfx  = pd.DataFrame([flatten_json(data)])
  dfn.append(dfx)

dfn   = pd.concat(dfn)
dfn['store'] = dfn['collection_featured_image_url'].str[:69]
dfn['contract'] = dfn['store'].str[-42:]
ctn = dfn[['collection_name','collection_slug','collection_stats_total_supply','contract']]
ctn = ctn.rename(columns={'collection_slug': 'slug','collection_name':'Collection Name','collection_stats_total_supply':'Supply'})
ctn

,Collection Name,slug,Supply,contract
0,Meebits,meebits,19999.0,None
0,CryptoPunks,cryptopunks,9999.0,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb
0,Forgotten Runes Wizards Cult,forgottenruneswizardscult,9448.0,nt.com/EYridn_9SjWo4zq9v9u8xuxuRmFis7251nt
0,MoonCats,acclimatedmooncats,18517.0,nt.com/kIfTnHDRFqRIOg1jkPoaD9oNwhOdLk2_0MO
0,ChainFaces,chainfaces,10000.0,com/opensea-static/featured-images/chainfa
...,...,...,...,...
0,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN
